In [ ]:
import pytz
import datetime
import numpy as np
import pandas as pd
from bokeh.models import NumberFormatter
from bokeh.resources import INLINE
import panel as pn

pn.extension()
pn.config.sizing_mode = 'stretch_width'

In [ ]:
ar = pytz.timezone('America/Argentina/Buenos_Aires')
now = datetime.datetime.now(ar)
dt_string = now.strftime("%d-%m-%Y %H:%M")

In [ ]:
header = pn.pane.Markdown(f"""
# COVID-19 -- Vacunas aplicadas en Argentina

[Volver a la página anterior](https://epassaro.github.io/covid-19/ar)

""", width=800, margin=(0, 0, 10, 0)
)

In [ ]:
footer = pn.pane.Markdown(f"""
Datos provistos por [Ministerio de Salud de la Nación](http://datos.salud.gob.ar/dataset/vacunas-contra-covid-19-dosis-aplicadas-en-la-republica-argentina)
e [INDEC](https://www.indec.gob.ar/indec/web/Nivel4-Tema-2-24-85).
El código fuente se distribuye en [GitHub](https://github.com/epassaro/covid-19) bajo la [GNU GPLv3 License](https://raw.githubusercontent.com/epassaro/covid-19/master/LICENSE).
<br> **Última actualización: ** _{dt_string}_ UTC−03:00. Powered by [Panel](https://panel.holoviz.org/index.html) and [GitHub Pages](https://pages.github.com/).

""", width=800, margin=(0, 0, 10, 0)
)

In [ ]:
fname = 'https://sisa.msal.gov.ar/datos/descargas/covid-19/files/Covid19VacunasAgrupadas.csv.zip'

In [ ]:
# Población estimada al 01/07/2021 según INDEC
pob_est_2021 = {'Argentina (Total país)': 45808747,
                'Buenos Aires': 17709598,
                'CABA': 3078836,
                'Catamarca': 418991,
                'Chaco': 1216247,
                'Chubut': 659284,
                'Corrientes': 1130320,
                'Córdoba': 3798261,
                'Entre Ríos': 1398510,
                'Formosa': 610019,
                'Jujuy': 779212,
                'La Pampa': 361394,
                'La Rioja': 398648,
                'Mendoza': 2010363,
                'Misiones': 1274992,
                'Neuquén': 672461,
                'Río Negro': 757052,
                'Salta': 1441988,
                'San Juan': 789489,
                'San Luis': 514610,
                'Santa Cruz': 374756,
                'Santa Fe': 3667505,
                'Santiago del Estero': 988245,
                'Tierra del Fuego': 177697,
                'Tucumán': 1791571}

In [ ]:
df = pd.read_csv(fname)

In [ ]:
df = df.drop(columns='jurisdiccion_codigo_indec')
df = df.rename(columns={'jurisdiccion_nombre': 'Jurisdicción', 
                        'vacuna_nombre': 'Vacuna', 
                        'primera_dosis_cantidad': 'Primera dosis',
                        'segunda_dosis_cantidad': 'Segunda dosis'})

In [ ]:
vacunas_nombres = df['Vacuna'].unique().tolist()
vacunas_nombres

In [ ]:
nuevos_nombres = ['AstraZeneca', 'Covishield', 'Sinopharm', 'Sputnik V']

In [ ]:
map_nombres = dict(zip(vacunas_nombres, nuevos_nombres))

In [ ]:
df = df.replace(map_nombres)

In [ ]:
df_tot = df.groupby('Vacuna').sum().reset_index()
df_tot['Jurisdicción'] = 'Argentina (Total país)'

In [ ]:
df = pd.concat([df_tot, df])
df

In [ ]:
pop_df = pd.DataFrame.from_records(zip(pob_est_2021.keys(), pob_est_2021.values()), columns=['Jurisdicción', 'Población'])
pop_df = pop_df.set_index('Jurisdicción')

In [ ]:
df = df.set_index(['Jurisdicción', 'Vacuna']).join(pop_df)

In [ ]:
df['Primera dosis (%)'] = df['Primera dosis'] / df['Población']
df['Segunda dosis (%)'] = df['Segunda dosis'] / df['Población']
df = df[['Primera dosis', 'Primera dosis (%)', 'Segunda dosis', 'Segunda dosis (%)' ]]

In [ ]:
formatter = NumberFormatter(format='0.00%') 

df_widget = pn.widgets.DataFrame(df,
                                 hierarchical=True, 
                                 aggregators={'Jurisdicción': 'sum', 
                                              'Vacuna': 'sum',
                                             },
                                 frozen_rows=1,
                                 formatters={'Primera dosis (%)': formatter,
                                             'Segunda dosis (%)': formatter},
                                 height=580,
                                 autosize_mode='fit_columns',
                                 margin=(0, 0, 10, 0),  #top, right, bottom, left
                                 )

In [ ]:
pn.Column(header, 
          df_widget,
          footer,
          margin=(20, 40, 0, 40)
         ).save('index.html', 
                title='COVID-19 - Vacunas aplicadas en Argentina', 
                resources=INLINE)